In [6]:
! pip install cvxpy

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.1 MB 330.3 kB/s eta 0:00:04
   -- ------------------------------------- 0.1/1.1 MB 469.7 kB/s eta 0:00:03
   ---- ----------------------------------- 0.1/1.1 MB 656.4 kB/s eta 0:00:02
   -------- ------------------------------- 0.2/1.1 MB 1.1 MB/s eta 0:00:01
   ------------ --------------------------- 0.3/1.1 MB 1.2 MB/s eta 0:00:01
   ------------------ --------------------- 0.5/1.1 MB 1.7 MB/s eta 0:00:01
   ------------------------------------ --- 1.0/1.1 MB 2.9 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 2.7 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 2.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/736.4 kB ? eta -:--:--
   --------------------------------------  727.0/736.4 kB 23.1 MB/s eta 0:00:01
   ------------------

In [9]:
! pip install pulp

   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/17.7 MB 222.6 kB/s eta 0:01:20
   ---------------------------------------- 0.0/17.7 MB 262.6 kB/s eta 0:01:08
   ---------------------------------------- 0.1/17.7 MB 393.8 kB/s eta 0:00:45
   ---------------------------------------- 0.1/17.7 MB 554.9 kB/s eta 0:00:32
    --------------------------------------- 0.2/17.7 MB 860.2 kB/s eta 0:00:21
    --------------------------------------- 0.3/17.7 MB 864.9 kB/s eta 0:00:21
   - -------------------------------------- 0.5/17.7 MB 1.5 MB/s eta 0:00:12
   - -------------------------------------- 0.6/17.7 MB 1.6 MB/s eta 0:00:11
   -- ------------------------------------- 1.1/17.7 MB 2.4 MB/s eta 0:00:07
   --- ------------------------------------ 1.6/17.7 MB 3.2 MB/s eta 0:00:06
   ---- ----------------------------------- 2.2/17.7 MB 4.1 MB/s eta 0:00:04
 

In [10]:
import numpy as np
from scipy.optimize import linprog
import cvxpy
from pulp import *

In [2]:
values = [4, 2, 1, 7, 3, 6] #стоимости товаров
weights = [5, 9, 8, 2, 6, 5] #вес товаров
C = 15 #вместимость сумки
n = 6 #количество товаров

c = - np.array(values) #изменяем знак, чтобы перейти от задачи максимизации к задаче минимизации
A = np.array(weights)  #конвертируем список с весами в массив
A = np.expand_dims(A, 0) #преобразуем размерность массива
b = np.array([C]) #конвертируем вместимость в массив

In [4]:
linprog(c=c, A_ub=A, b_ub=b, method='highs')

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -52.5
              x: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00  0.000e+00
                   0.000e+00]
            nit: 0
          lower:  residual: [ 0.000e+00  0.000e+00  0.000e+00  7.500e+00
                              0.000e+00  0.000e+00]
                 marginals: [ 1.350e+01  2.950e+01  2.700e+01  0.000e+00
                              1.800e+01  1.150e+01]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00]
                 marginals: [-3.500e+00]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [8]:
x = cvxpy.Variable(shape=n, integer=True)
constraint = cvxpy.sum(cvxpy.multiply(A, x)) <= C
x_positive = x >= 0
total_value = cvxpy.sum(cvxpy.multiply(x, c))

problem = cvxpy.Problem(
    cvxpy.Minimize(total_value), constraints=[constraint, x_positive]
)

problem.solve()
x.value

array([-0., -0., -0.,  7., -0.,  0.])

In [12]:
problem = LpProblem('Производство машин', LpMaximize)
A = LpVariable('Автомобиль A', lowBound=0 , cat=LpInteger)
B = LpVariable('Автомобиль B', lowBound=0 , cat=LpInteger)
#Целевая функция
problem += 20000*A + 45000*B
#Ограничения
problem += 4*A + 5*B <= 30
problem += 3*A + 6*B <=30
problem += 2*A + 7*B <=30
problem.solve()
print("Количество автомобилей модели А:", A.varValue)
print("Количество автомобилей модели В:", B.varValue)
print("Суммарный доход: ", value(problem.objective))
#Количество автомобилей модели А:  1.0
#Количество автомобилей модели В:  4.0
#Суммарный доход:  200000.0

Количество автомобилей модели А: 1.0
Количество автомобилей модели В: 4.0
Суммарный доход:  200000.0
